In [ ]:
import pulp
from pulp import PULP_CBC_CMD
import tsplib95

problem = tsplib95.load('br17.atsp')

G = problem.get_graph()

In [27]:
from mtz_pulp import add_subtour_constraints

n = len(G.nodes)
nodes = list(G.nodes)

model = pulp.LpProblem("TSP_MTZ", pulp.LpMinimize)

# Define binary decision variables x[i,j] for edges i->j
x = {(i, j): pulp.LpVariable(f"x_{i}_{j}", cat=pulp.LpBinary)
     for i in nodes for j in nodes if i != j}

# Extract costs from the graph 
costs = nx.get_edge_attributes(G, 'weight')

# Objective: Minimize sum of cost * x[i,j]
model += pulp.lpSum(costs[(i, j)] * x[(i, j)]
                    for i in nodes for j in nodes if i != j), "Minimize_Cost"

for i in nodes:
    # Outflow constraint for city i
    model += pulp.lpSum(x[(i, j)] for j in nodes if j != i) == 1, f"Outflow_{i}"

    # Inflow constraint for city i
    model += pulp.lpSum(x[(j, i)] for j in nodes if j != i) == 1, f"Inflow_{i}"

# Define u
u = {i: pulp.LpVariable(f"u_{i}", lowBound=0, upBound=n-1, cat=pulp.LpInteger)
     for i in nodes}

# Add MTZ constraints
for i in range(1, n):  
    for j in nodes:
        if i != j:
            model += u[i] - u[j] + n*x[(i, j)] <= n-1, f"MTZ_{i}_{j}"
# model.solve(PULP_CBC_CMD(gapRel = 0.02))
model.solve(pulp.GUROBI_CMD())

Set parameter Username
Set parameter LicenseID to value 2594484
Set parameter LogFile to value "gurobi.log"
Using license file /Library/gurobi1200/gurobi.lic
Academic license - for non-commercial use only - expires 2025-12-03

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.1.0 24B91)
Copyright (c) 2024, Gurobi Optimization, LLC

Read LP format model from file /var/folders/tf/kyhcn0rj15722n_6n2h54bcc0000gn/T/37c1ff29eac2430ab650262541491b25-pulp.lp
Reading time = 0.00 seconds
Minimize_Cost: 290 rows, 289 columns, 1312 nonzeros

Using Gurobi shared library /Users/chuck/miniforge3/envs/CS632/lib/libgurobi120.dylib

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 290 rows, 289 columns and 1312 nonzeros
Model fingerprint: 0x02eebc7e
Variable types: 0 continuous, 289 integer (272 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+00, 7e+01]
  Bounds ra

1

In [28]:
print("Cost:", pulp.value(model.objective))

Cost: 39.0


In [29]:
from pulp import value

# Ensure model is solved
if value(model.status) == 1:
    # Extract solution values: which edges are chosen?
    chosen_edges = [(i, j) for (i, j) in x if value(x[(i, j)]) == 1]

    # Reconstruct tour
    # Start from node 0 (or any arbitrary node)
    start_node = 0
    tour = [start_node]
    current_node = start_node

    # There are n nodes, so we add n-1 more edges
    for _ in range(n - 1):
        # Find the next node that current_node connects to
        for (u, v) in chosen_edges:
            if u == current_node:
                tour.append(v)
                current_node = v
                break
tour

[0, 11, 7, 8, 16, 3, 4, 6, 14, 15, 5, 1, 10, 12, 9, 2, 13]